# Behaviour Cloning

Train a Neural Network to drive a car.

This time I use transfer learning to do feature extraction.

## Feature Extraction using Transfer Learning
To extract features from all images.

In [1]:
'''import the libs and load the test image'''
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

#image=mpimg.imread("test_image.jpg")
#plt.imshow(image)

In [2]:
'''import the libs for machine learning'''
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from keras.layers.core import Dense, Dropout, Activation,Flatten
from keras.optimizers import Adam
from keras.regularizers import l2
from sklearn.cross_validation import train_test_split

Using TensorFlow backend.
/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
ImgSize=(320,160)

In [4]:
# TODO: preprocess the image.
# Crop+Resize
# Then flip for training as augmentation
def preprocess(img, flip=False):
    global ImgSize
    '''
    parameters:
    img: a img matrix
    '''
    # TODO: resize
    img = cv2.resize(img,(ImgSize[0],ImgSize[1]))
    # TODO: crop
    height = img.shape[0]
    img = img[height/4:height,:]
    
    # Flip
    if flip:
        img = cv2.flip(img,1)
    
    # Normalization
    img = img/127.5 - 1
        
    
    return img
    
    

In [5]:
# TODO: preprocess pipelines
def read_and_prep(filename):
    img = plt.imread("data/"+filename)
    return preprocess(img)

def read_prep_flip(filename):
    img = plt.imread("data/"+filename)
    return preprocess(img,True)

## Process Training&Testing Data

Create a generator for training and testing the neural network.

In [6]:
'''read the dataset'''
import pandas as pd
data=pd.read_csv("data/driving_log.csv")

In [7]:
'''fix the bug of record dataframe that the 1st char of filename is space'''
def record_fix(filename):
    if isinstance(filename,str):
        if filename[0]==" ":
            return filename[1:]
    return filename
        

In [8]:
'''fix them'''
data=data.applymap(record_fix)

In [9]:
# TODO: make a better generator, including image preprocess,
# and ensure that image features extracted are consistent.

# When using left or right, +/- 0.25
# Note: this idea came from vivek

def data_preprocess(X,y):
    '''
    parameters:
    X,y: in arrays
    '''
    # TODO:apply
    X_orig = np.stack(X.apply(read_and_prep).values)
    X_flip = np.stack(X.apply(read_prep_flip).values)
    y = y.values
    
    X_all = np.vstack([X_orig,X_flip])
    y_all = np.hstack([y,-y])
    
    return X_all,y_all


def generator(data):
    global Batch_size
    i = np.random.randint(len(data)) #init the index
    while 1:
        if i+Batch_size>data.shape[0]: # if go beyound, return to 0
            i = 0
        
        part = data.iloc[i:i+Batch_size,:]
        
        # TODO: to array
        X_center=part["center"]
        X_left=part["left"]
        X_right=part["right"]
        
        y_center=part["steering"]
        y_left=y_center+0.25
        y_right=y_center-0.25
        
        
        # TODO: center image getting out with flip
        yield data_preprocess(X_center,y_center)

        # TODO: left img gettin out with flip
        yield data_preprocess(X_left,y_left)

        # TODO: right img gettin out with flip
        yield data_preprocess(X_right,y_right)
    
    
    


## Train the output linear regression unit

In [10]:
'''# train test split'''
train,test=train_test_split(data,test_size=0.1,random_state=2016)

In [11]:
# test the tts
train.head()

,center,left,right,steering,throttle,brake,speed
7139,IMG/center_2016_12_01_13_45_10_891.jpg,IMG/left_2016_12_01_13_45_10_891.jpg,IMG/right_2016_12_01_13_45_10_891.jpg,-0.145206,0.985533,0.0,30.18542
780,IMG/center_2016_12_01_13_33_57_313.jpg,IMG/left_2016_12_01_13_33_57_313.jpg,IMG/right_2016_12_01_13_33_57_313.jpg,0.000000,0.985533,0.0,30.18713
7794,IMG/center_2016_12_01_13_46_17_277.jpg,IMG/left_2016_12_01_13_46_17_277.jpg,IMG/right_2016_12_01_13_46_17_277.jpg,0.000000,0.985533,0.0,30.18667
5601,IMG/center_2016_12_01_13_42_34_881.jpg,IMG/left_2016_12_01_13_42_34_881.jpg,IMG/right_2016_12_01_13_42_34_881.jpg,0.000000,0.985533,0.0,30.18600
5237,IMG/center_2016_12_01_13_41_57_922.jpg,IMG/left_2016_12_01_13_41_57_922.jpg,IMG/right_2016_12_01_13_41_57_922.jpg,0.000000,0.985533,0.0,30.18625


In [12]:
test.head()

,center,left,right,steering,throttle,brake,speed
4365,IMG/center_2016_12_01_13_40_00_721.jpg,IMG/left_2016_12_01_13_40_00_721.jpg,IMG/right_2016_12_01_13_40_00_721.jpg,0.00000,0.985533,0.0,30.186540
60,IMG/center_2016_12_01_13_32_44_368.jpg,IMG/left_2016_12_01_13_32_44_368.jpg,IMG/right_2016_12_01_13_32_44_368.jpg,0.00000,0.985533,0.0,12.316180
18,IMG/center_2016_12_01_13_31_14_602.jpg,IMG/left_2016_12_01_13_31_14_602.jpg,IMG/right_2016_12_01_13_31_14_602.jpg,0.00000,0.000000,0.0,1.230361
7071,IMG/center_2016_12_01_13_45_03_987.jpg,IMG/left_2016_12_01_13_45_03_987.jpg,IMG/right_2016_12_01_13_45_03_987.jpg,0.00000,0.985533,0.0,30.186630
1033,IMG/center_2016_12_01_13_34_22_936.jpg,IMG/left_2016_12_01_13_34_22_936.jpg,IMG/right_2016_12_01_13_34_22_936.jpg,0.06176,0.985533,0.0,30.186580


In [13]:
'''# import useful libs for modeling and training'''
from keras.models import Sequential,Model
from keras.layers import Input

In [14]:
# TODO: 2 hidden layer regression model
def two_hidden_inception_large():
    img_input = Input(shape=(ImgSize[1]*0.75,ImgSize[0],3))
    base = InceptionV3(weights="imagenet",input_tensor=img_input,include_top=False)
    flatten = Flatten()(base.output)
    flatten_dropout = Dropout(0.25)(flatten)
    fc1 = Dense(1024,activation="relu")(flatten_dropout)
    fc1_dropout = Dropout(0.25)(fc1)
    fc2 = Dense(512,activation="relu")(fc1_dropout)
    fc2_dropout = Dropout(0.25)(fc2)
    steer = Dense(1)(fc2_dropout)
    
    model = Model(input=base.input,output=steer)
    model.compile("adam","mse")
    
    return model

In [15]:
# TODO: 2 hidden layer regression model
def two_hidden_inception():
    img_input = Input(shape=(ImgSize[1]*0.75,ImgSize[0],3))
    base = InceptionV3(weights="imagenet",input_tensor=img_input,include_top=False)
    flatten = Flatten()(base.output)
    flatten_dropout = Dropout(0.25)(flatten)
    fc1 = Dense(128,activation="relu")(flatten_dropout)
    fc1_dropout = Dropout(0.5)(fc1)
    fc2 = Dense(64,activation="relu")(fc1_dropout)
    fc2_dropout = Dropout(0.5)(fc2)
    steer = Dense(1)(fc2_dropout)
    
    model = Model(input=base.input,output=steer)
    model.compile("adam","mse")
    
    return model
    

In [16]:
def two_hidden_vgg():
    img_input = Input(shape=(ImgSize[1]*0.75,ImgSize[0],3))
    base = VGG16(weights="imagenet",input_tensor=img_input,include_top=False)
    flatten = Flatten()(base.output)
    flatten_dropout = Dropout(0.5)(flatten)
    fc1 = Dense(256,activation="relu")(flatten_dropout)
    fc1_dropout = Dropout(0.5)(fc1)
    fc2 = Dense(128,activation="relu")(fc1_dropout)
    fc2_dropout = Dropout(0.75)(fc2)
    steer = Dense(1)(fc2_dropout)
    
    model = Model(input=base.input,output=steer)
    model.compile("adam","mse")
    
    return model


In [17]:
from keras.models import Sequential
from keras.layers import Convolution2D
def end2end():
    model = Sequential()
    model.add(Convolution2D(24,5,5,border_mode='valid',activation='relu', subsample=(2,2),input_shape=(ImgSize[1]*3/4,ImgSize[0],3)))
    model.add(Convolution2D(36,5,5,border_mode='valid', activation='relu', subsample=(2,2)))
    model.add(Convolution2D(48,5,5,border_mode='valid', activation='relu', subsample=(2,2)))
    model.add(Convolution2D(64,3,3,border_mode='valid', activation='relu', subsample=(1,1)))
    model.add(Convolution2D(64,3,3,border_mode='valid', activation='relu', subsample=(1,1)))
    model.add(Flatten())
    model.add(Dense(1164, activation='relu'))
    model.add(Dropout(.25))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer = 'adam', loss = 'mse')
    
    return model
    

In [18]:
#model_large = two_hidden_inception_large()
#model_inception = two_hidden_inception()
#model_vgg = two_hidden_vgg()

In [26]:
'''# parameters'''
Samples_per_iter= 60000 # TOTUNE
Num_val_samples=1024 # TOTUNE
Iters=24
Batch_size=128

In [20]:
'''training function'''
def training(model,train_data):
    global test #testing dataset
    model.fit_generator(
        generator(train_data), 
        nb_epoch=Iters,
        max_q_size=3,
        samples_per_epoch=Samples_per_iter,
        validation_data=generator(test), # use 3 for vali each time
        # TODO: change this line
        nb_val_samples=Num_val_samples
    )

In [21]:
'''evaluation'''
def evaluate(model,num_test_samples):
    global test
    return model.evaluate_generator(
                        generator=generator(test),
                        val_samples=num_test_samples)


In [22]:
model_e2e = end2end()
training(model_e2e,train)

Epoch 1/12


/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel/__main__.py:14: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


59904/60000 [============================>.] - ETA: 0s - loss: 0.0038

/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/keras/engine/training.py:1569: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


60032/60000 [==============================] - 206s - loss: 0.0038 - val_loss: 0.0284
Epoch 2/12
60032/60000 [==============================] - 177s - loss: 2.6937e-04 - val_loss: 0.0299
Epoch 3/12
60032/60000 [==============================] - 176s - loss: 1.8273e-04 - val_loss: 0.0293
Epoch 4/12
60032/60000 [==============================] - 176s - loss: 1.3710e-04 - val_loss: 0.0276
Epoch 5/12
60032/60000 [==============================] - 177s - loss: 1.0795e-04 - val_loss: 0.0274
Epoch 6/12
60032/60000 [==============================] - 176s - loss: 1.0004e-04 - val_loss: 0.0273
Epoch 7/12
60032/60000 [==============================] - 176s - loss: 8.3615e-05 - val_loss: 0.0270
Epoch 8/12
60032/60000 [==============================] - 176s - loss: 7.2730e-05 - val_loss: 0.0277
Epoch 9/12
60032/60000 [==============================] - 175s - loss: 6.9334e-05 - val_loss: 0.0271
Epoch 10/12
60032/60000 [==============================] - 171s - loss: 6.1722e-05 - val_loss: 0.0269
Epoc

In [27]:
from keras.models import load_model
model = load_model("model_e2e.h5")

In [ ]:
training(model,train)

Epoch 1/24


/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel/__main__.py:14: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


 8192/60000 [===>..........................] - ETA: 143s - loss: 0.0380

In [ ]:
training(model_large,train)

Epoch 1/5


/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel/__main__.py:14: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


48128/48216 [============================>.] - ETA: 1s - loss: 0.0082   

/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/keras/engine/training.py:1569: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


48256/48216 [==============================] - 861s - loss: 0.0081 - val_loss: 0.0229
Epoch 2/5
48256/48216 [==============================] - 861s - loss: 0.0053 - val_loss: 0.0229

In [28]:
training(model_e2e,train)

Epoch 1/5


/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel/__main__.py:14: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


48256/48256 [==============================] - 142s - loss: 1.9737e-04 - val_loss: 0.0110
Epoch 2/5
48256/48256 [==============================] - 141s - loss: 1.4974e-04 - val_loss: 0.0114
Epoch 3/5
48256/48256 [==============================] - 140s - loss: 1.2378e-04 - val_loss: 0.0113
Epoch 4/5
48256/48256 [==============================] - 141s - loss: 1.1324e-04 - val_loss: 0.0114
Epoch 5/5
48256/48256 [==============================] - 142s - loss: 1.0233e-04 - val_loss: 0.0116


In [ ]:
training(model_inception,train)

In [ ]:
training(model_vgg,train)

In [21]:
# evaluate regression model
evaluate(model_inception_large,1500),evaluate(model_inception,1500),evaluate(model_vgg,1500)

/Users/jhsiao/anaconda/envs/py3/lib/python3.5/site-packages/ipykernel/__main__.py:14: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


107.91423950195312

## Output the architecture and weights

In [23]:
model_e2e.save("model_e2e.h5")
#model_inception.save("model_inception.h5")
#model_vgg.save("model_vgg.h5")

In [ ]:
model_e2e.save("model_e2e_24.h5")